<div><img style="float: right; width: 120px; vertical-align:middle" src="https://www.upm.es/sfs/Rectorado/Gabinete%20del%20Rector/Logos/EU_Informatica/ETSI%20SIST_INFORM_COLOR.png" alt="ETSISI logo" />


# Creando un chatbot simple<a id="top"></a>

<i><small>Autor: Alberto Díaz Álvarez<br>Última actualización: 2023-03-28</small></i></div>
                                                  

***

## Introducción

Los _chatbots_ son sistemas informáticos diseñados para simular una conversación humana y responder preguntas de manera automatizada. Estos sistemas han ganado popularidad en los últimos años debido a su capacidad para proporcionar asistencia al cliente las 24 horas del día, los 7 días de la semana, y su potencial para mejorar la eficiencia de los negocios al automatizar tareas rutinarias.

Los chatbots se han vuelto especialmente útiles en la industria del servicio al cliente, donde pueden ayudar a las empresas a reducir los tiempos de espera y mejorar la satisfacción del cliente.

## Objetivos

El objetivo de este proyecto es diseñar y desarrollar un chatbot simple siguiedo la arquitectura _seq2seq_. Este modelo, también llamado codificador-decodificador, utiliza la (corta) memoria a largo plazo que mantienen unidades como LSTM o GRU para generar una secuencia de salida a partir de una secuencia de entrada tras haber sido entrenado con un corpus suficientemente grande de secuencias origen y sus correspondientes secuencias de destino.

## Imports y configuración

A continuación importaremos las librerías que se usarán a lo largo del notebook.

In [1]:
import glob
import os
import shutil
import yaml

import numpy as np
import requests
import tensorflow as tf

***

## Constantes previas

Vamos a comenzar definiendo una serie de constantes que usaremos a lo largo del _notebook_.

Comenzamos por las etiquetas referentes a comienzo y fin de frase:

In [2]:
TOKEN_SOS = '#sos#'
TOKEN_EOS = '#eos#'

Definimos también el número de características de salida del embedding, el máximo de longitud de secuencia y el tamaño de nuestro vocabulario de palabras.

In [3]:
EMBEDDING_DIM = 50
MAX_SEQUENCE_LEN = 20
VOCABULARY_SIZE = 2500

Por último, indicamos las "unidades" (neuronas) de salida de nuestra unidad recurrente.

In [4]:
UNITS = 256

## Carga y preprocesamiento

Vamos a descargar una serie de pares de pregunta-respuesta para que nuestro _chatbot_ aprenda de ellas. Vamos a hacerlo en español, por cambiar un poco.

Para ello descargaremos el repositorio [ChatterBot Language Training Corpus](https://github.com/gunthercox/chatterbot-corpus) de GitHub, el cual contiene varios _corpus_ en varios idiomas.

In [5]:
CORPUS_URL = 'https://github.com/gunthercox/chatterbot-corpus/archive/master.zip'
CORPUS_FILE = 'tmp/master.zip'

# Creamos el directorio temporal si no existiese ya
TMP_DIR = 'tmp/'
if not os.path.isdir(TMP_DIR):
    os.makedirs(TMP_DIR)

# Descargamos todo el repositorio
if not os.path.exists(CORPUS_FILE):
    print('Downloading corpus ... ', end='')
    with open(CORPUS_FILE, 'wb') as f:
        r = requests.get(CORPUS_URL, allow_redirects=True)
        f.write(r.content)
    print('OK')

# Lo descomprimimos en el directorio temporal
print('Unpacking corpus ... ', end='')
shutil.unpack_archive(CORPUS_FILE, TMP_DIR)
print('OK')

Unpacking corpus ... OK


Con el dataset descargado, recogemos todas las conversaciones de la sección en español. La verdad es que las conversaciones no son muy para tirar cohetes, pero bueno, a ver qué sale.

In [6]:
print('Loading ... ', end='')
shutil.unpack_archive(CORPUS_FILE, TMP_DIR)
chats = []
for path in glob.glob(f'{TMP_DIR}chatterbot-corpus-master/chatterbot_corpus/data/spanish/*.yml'):
    with open(path) as f:
        chats.extend(c for c in yaml.safe_load(f).get('conversaciones', []))
print(f'{len(chats)} conversations loaded')

Loading ... 50 conversations loaded


Bien, ya tenemos unas cuantas conversaciones que se componen de "preguntas y "respuestas" (bueno, no tienen por qué ser técnicamente preguntas y respuestas, pero bueno, el toma y daca de una conversación).

Ahora lo separaremos en tres conjuntos:

- `questions`: El de preguntas, sin nada especial.
- `answers`: El de respuestas, con el que alimentaremos la parte decodificadora de nuestra red, y que por tanto requerirá dos etiquetas al principio y al final para que denoten el comienzo y el fin de una frase
- `answers_no_start`: El de respuestas con el que contrastaremos la respuesta dada por el decodificador, y que por tanto sólo requerirá una etiqueta al final para que denote el fin de una frase.

In [7]:
questions = []
answers = []
answers_no_start = []

print('Processing conversations ... ', end='')
for chat in chats:
    for q, a in zip(chat[:-1], chat[1:]):
        questions.append(q)
        answers.append(f'{TOKEN_SOS} {a} {TOKEN_EOS}')
        answers_no_start.append(f'{a} {TOKEN_EOS}')
print(f'{len(questions)} answers and questions loaded')

Processing conversations ... 212 answers and questions loaded


No es mucho, pero bueno, es algo por lo que comenzar.

Una vez tenemos nuestras frases, vamos a crear el componente que traducirá estas fsentencias a secuencias de tamaño fijo de enteros. Para esto usaremos una capa denominada `TextVectorization`, la cual hace la transformación automática (según la hayamos configurado, claro) de textos a secuencias de enteros, y que es directamente encajable en el _pipeline_ de un modelo.

In [8]:
# Creamos la capa de vectorización (frase -> secuencia de enteros)
def standardize(inputs):
    inputs = tf.strings.regex_replace(inputs, r'[!"$%&()\*\+,-\./:;<=>?@\[\\\]^_`{|}~\']', "")
    return tf.strings.lower(inputs)

vectorization_layer = tf.keras.layers.TextVectorization(
    max_tokens=VOCABULARY_SIZE,
    output_sequence_length=MAX_SEQUENCE_LEN,
    standardize=standardize,
    name='vectorization',
)

# Lo inicializamos con el vocabulario de nuestro dataset. Aunque aquí
# no es necesario, lo cargamos en batch para ilustrar cómo funciona.
# Esto nos permitiría trabajar con datasets muy grandes.
text_dataset = tf.data.Dataset.from_tensor_slices(questions + answers)
vectorization_layer.adapt(text_dataset.batch(64))

Y vamos a aprovechar que hemos creado nuestra capa de vectorización para transformar a vectores el conjunto de salida con el que compararemos la salida de nuestro modelo.

In [9]:
answers_no_start = vectorization_layer(answers_no_start).numpy()
print(answers_no_start)

[[  9 197   4 ...   0   0   0]
 [  4  23  38 ...   0   0   0]
 [  8 304 150 ...   0   0   0]
 ...
 [ 15 396 100 ...   0   0   0]
 [ 17 395   4 ...   0   0   0]
 [ 17 194  45 ...   0   0   0]]


Por último, al igual que hemos creado la capa de vectorización, crearemos también la capa de _embedding_ que usaremos tanto en la parte codificadora como en la decodificadora de nuestro modelo seq2seq. Igual que la estamos creando, podemos hacer uso de un embedding ya existente como GLoVe (y lo mismo esto no lo leéis y lo he cambiado para tirar de una implementación de GLoVe en español).

In [10]:
embedding_layer = tf.keras.layers.Embedding(
    VOCABULARY_SIZE + 1, # El 0 está vetado, por eso es +1
    EMBEDDING_DIM,
    mask_zero=True,      # 0 es padding (no se debe usar en el vocabulario)
    name='embedding',
)

## Creación del modelo

Ya hemos visto en teoría que el esquema _seq2seq_ es un problema tipo _many-to-many_. De alguna manera tenemos que conseguir que nuestro modelo aprenda las relaciones existentes entre las palabras de nuestro texto junto con una codificación que tenga en cuenta estas relaciones y el contexto de las preguntas y respuestas.

Lo que haremos será entrenar simultáneamente dos capas GRU. Concretamente entrenaremos la primera para las preguntas y luego utilizaremos sus pesos como estado inicial para entrenar la segunda para las respuestas.

La primera capa GRU será el **codificador**, esto es, el procesamiento de la entrada para devolver una codificación de esta en su estado interno.

In [11]:
encoder_gru = tf.keras.layers.GRU(UNITS, return_state=True, name='Encoder-GRU')

La entrada a esta capa será una secuencia ya vectorizada y convertida a vectores de palabras.

In [12]:
encoder_inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string, name='Question')
encoder_inputs_word_vectors = embedding_layer(vectorization_layer(encoder_inputs))

encoder_output, encoder_state = encoder_gru(encoder_inputs_word_vectors)

La segunda capa GRU será la correspondiente al **decodificador**. Tomará el estado como contexto para predecir las siguientes palabras de la secuencia objetivo. Devolverá tantas salidas como elementos tenga la secuencia de entrada, básicamente para poder entrenala.

In [13]:
decoder_gru = tf.keras.layers.GRU(UNITS, return_state=True, return_sequences=True, name='Decoder-GRU')

El decodificador se conectará también a una secuencia de entrada (para hacer _teacher enforcing_) ya vectorizada y pasada a vectores de palabras, y a una capa densa de salida que hará la clasificación a la palabra más probable.

In [14]:
decoder_inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string, name='Answer')
decoder_inputs_word_vectors = embedding_layer(vectorization_layer(decoder_inputs))

decoder_output, _ = decoder_gru(decoder_inputs_word_vectors, initial_state=[encoder_state])
decoder_dense = tf.keras.layers.Dense(VOCABULARY_SIZE, activation='softmax', name='Decoder-classifier')
decoder_output = decoder_dense(decoder_output)

Hay que prestar especial atención a que la capa `Decoder-GRU` recibe como estado inicial el estado de la capa `Encoder-GRU`.

Ahora ya podemos crear el modelo especificando entradas y salidas.

In [15]:
model = tf.keras.models.Model([encoder_inputs, decoder_inputs], [decoder_output])
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='sparse_categorical_crossentropy')
model.summary()
tf.keras.utils.plot_model(model)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Answer (InputLayer)            [(None, 1)]          0           []                               
                                                                                                  
 Question (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 vectorization (TextVectorizati  (None, 20)          0           ['Question[0][0]',               
 on)                                                              'Answer[0][0]']                 
                                                                                                  
 embedding (Embedding)          (None, 20, 50)       125050      ['vectorization[0][0]',      

Dado que la salida de nuestro modelo (la salida del _decoder_) es un _softmax_ de 2500 palabras, usamos `sparse_categorical_crossentropy` para utilizar el índice de la palabra en lugar de pasar a una codificación _one-hot_ todas las labels.

Ahora vamos a entrenar el modelo

In [16]:
model.fit([np.array(questions), np.array(answers)], np.array(answers_no_start), epochs=10)

Epoch 1/10
7/7 [==============================] - 6s 61ms/step - loss: 7.8057
Epoch 2/10
7/7 [==============================] - 0s 66ms/step - loss: 7.7134
Epoch 3/10
7/7 [==============================] - 1s 83ms/step - loss: 6.6458
Epoch 4/10
7/7 [==============================] - 1s 81ms/step - loss: 5.5695
Epoch 5/10
7/7 [==============================] - 1s 81ms/step - loss: 5.2433
Epoch 6/10
7/7 [==============================] - 1s 83ms/step - loss: 5.1109
Epoch 7/10
7/7 [==============================] - 1s 81ms/step - loss: 5.0304
Epoch 8/10
7/7 [==============================] - 1s 78ms/step - loss: 4.9735
Epoch 9/10
7/7 [==============================] - 1s 79ms/step - loss: 4.9138
Epoch 10/10
7/7 [==============================] - 1s 80ms/step - loss: 4.8814


Una vez entrenado el modelo general, extraeremos las capas entrenadas a dos componentes. Primero, el **_encoder_**, el más sencillo. Su entrada será una pregunta/frase y su salida será el estado interno de la neurona.

In [18]:
!pip install pydot

Defaulting to user installation because normal site-packages is not writeable


In [19]:
!pip install graphviz

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/47.0 kB ? eta -:--:--
     ---------------------------------------- 47.0/47.0 kB 2.5 MB/s eta 0:00:00


In [20]:
encoder_model = tf.keras.models.Model(encoder_inputs, encoder_state)
tf.keras.utils.plot_model(encoder_model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


Segundo, el **_decoder_**, que tomará dos entradas: el estado del _encoder_ (el espacio latente de la pregunta) y la palabra anterior (o el token _start of sequence_ si no la hay), con la que generaremos la nueva palabra.

In [21]:
decoder_input_state = tf.keras.layers.Input(shape=(UNITS,), name='Input-state')
decoder_output, decoder_state = decoder_gru(decoder_inputs_word_vectors, initial_state=[decoder_input_state])
decoder_output = decoder_dense(decoder_output)
decoder_model = tf.keras.models.Model(
    [decoder_inputs, decoder_input_state],
    [decoder_output, decoder_state],
)
tf.keras.utils.plot_model(decoder_model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


Ahora ya podemos hablar con el chatbot. El proceso es el siguiente:

1. Tomamos una pregunta como entrada y predecimos los valores de estado utilizando el _encoder_.
1. Establecemos los valores de estado del _encoder_ en el _decoder_.
1. Establecemos el valor `TOKEN_SOS` en la entrada del decoder y obtenemos el valor de salida $O$.
1. Mientras $O$ no sea el token `TOKEN_EOS` o lleguemos a la longitud máxima de secuencia
    1. Añadimos el valor de salida $O$ a nuestra secuencia resultado
    2. Reemplazamos la entrada del decoder por $O$ y obtenemos el nuevo $O$ de la salida del decoder

Lo implementaremos en una clase que denominaremos FedericoTalker porque sí.

In [22]:
class FedericoTalker:
    def __init__(self, encoder, decoder):
        self.encoder = encoder
        self.decoder = decoder

    def tell(self, message, max_len=None):
        # Codificamos la pregunta
        state = encoder_model.predict([message], verbose=0)
        # Vamos generando palabras hasta que terminemos la frase
        response = [TOKEN_SOS]
        while response[-1] != TOKEN_EOS and len(response) < (max_len or np.inf):
            output, state = decoder_model.predict([np.array([[response[-1]]]), state], verbose=0)
            token = np.argmax(output[0,-1])
            response.append(vectorization_layer.get_vocabulary()[token])
        # Devolvemos la frase sin los tokens de comienzo y fin de frase
        return ' '.join(response[1:-1])

federico_talker = FedericoTalker(encoder_model, decoder_model)

Ahora podemos crear un nuevo _chatbot_ con nuestros _encoder_ y _decoder_ y charlar con él.

In [23]:
federico_talker.tell('¿Perdona?')

'no  no'

In [24]:
federico_talker.tell('Me apetece saberlo')

KeyboardInterrupt: 

In [25]:
federico_talker.tell('Creo que con tan poca conversación es difícil que me contestes a algo con sentido')

KeyboardInterrupt: 

## Conclusiones

La implementación de un chatbot basado en _seq2seq_ en español es una tarea algo compleja, pero mola ver cómo es capaz de responder (aunque sea de manera rudimentaria) a las preguntas que le hacemos.

Este tipo de modelo de redes neuronales recurrentes es capaz de generar respuestas relativamente coherentes y contextualizadas a partir de las preguntas que se le plantean. La clave para el éxito de la implementación radica en una adecuada selección y preparación de los datos de entrenamiento, así como en una arquitectura de red neuronal bien diseñada y ajustada. Además, el uso de técnicas como el modelo de atención puede mejorar significativamente la calidad de las respuestas generadas por el chatbot.

En general, la implementación de un chatbot basado en _seq2seq_ en español es una muy buena forma de experimentar con técnicas avanzadas de procesamiento del lenguaje natural y de construir un modelo capaz de comunicarse de manera efectiva con los usuarios.

***

<div><img style="float: right; width: 120px; vertical-align:top" src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" alt="Creative Commons by-nc-sa logo" />

[Volver al inicio](#top)

</div>